In [0]:
filePath = "/content/drive/My Drive/Yelp Project/data/"

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
import time
import numpy as np
import pandas as pd
import tensorflow as tf
import scipy.sparse as sp
import matplotlib.pyplot as plt
from sklearn.metrics import roc_auc_score, accuracy_score, mean_squared_error

pd.set_option('display.max_columns', None)
%matplotlib inline

In [0]:
businuessColumns = ['state', 'business_id'] #'review_count', 'stars', 
userColumns = ["user_id", "average_stars", "review_count"] #, "elite", "compliment_photos", "compliment_list", "compliment_funny", "compliment_plain", "compliment_note", "compliment_writer", "compliment_cute", "compliment_more", "compliment_hot", "compliment_profile", "compliment_cool", "useful", "fans", "funny", "cool", 
ratingsColumns = ["business_id", "stars", "user_id"] #"cool", "funny", "useful", 
targetColumn = "stars"

In [0]:
business = pd.read_csv(filePath + "business_cleaned.csv", usecols=businuessColumns)
user = pd.read_csv(filePath + 'user_cleaned.csv', usecols=userColumns)
ratings = pd.read_csv(filePath + 'review_cleaned_withTrainTest.csv')

In [7]:
print(business['business_id'].nunique())
print(business['business_id'].drop_duplicates().shape[0])

print(user['user_id'].nunique())
print(user['user_id'].drop_duplicates().shape[0])

158525
158525
818922
818922


In [8]:
print(ratings['business_id'].nunique())
print(ratings['business_id'].drop_duplicates().shape[0])

print(ratings['user_id'].nunique())
print(ratings['user_id'].drop_duplicates().shape[0])

9017
9017
48644
48644


In [9]:
ratings.head()

,user_id,date,funny,useful,review_id,text,business_id,stars,cool,TrainTest
0,BKcmc3Rj6L1EHeou_07IUw,2017-01-01 00:00:05,0,0,Mw89_xzMrWBPZJJkdGbnAA,Poor customer service. The only I gave this pl...,gBfPyzPRmeOaj3SdcIj0Rw,2.0,0,1
1,p4WztGxrJBrVWmt-xmY5Fg,2017-01-01 00:02:38,0,2,B55t0ioeFWWSlAQX4-0Q9Q,Came for brunch after hearing great reviews on...,xBgQ4T6mhS5VsmK9u6mBcg,4.0,0,1
2,8z-gipHHulR2HAeAE3pWpw,2017-01-01 00:03:16,0,2,OLeTF2dkoqeHo7qaErl2VA,We visited BWW to view the UFC 207 fight. This...,v8QuF4YX9Bb-qEtKLVPrmA,4.0,2,1
3,xRpAEg0M0azAPuKVg7WtgA,2017-01-01 00:04:18,4,1,GUsBYum0YCipOJ3HNyZ37g,Can't wait until I am 60 then I will get a car...,gjzQoi1AkRxYRl8-dFGGPQ,4.0,1,1
4,KCcGBudPGBLoviv5BbPiXQ,2017-01-01 00:05:19,0,0,-BlhsXgJFd8bd3_y71CkDw,I just love Nothing Bundt Cakes. They created ...,Ay7SK-nziJ7hnf6V_z6saA,5.0,0,1


In [0]:
ratings = pd.merge(ratings, business, how='left', left_on='business_id', right_on='business_id')
ratings = pd.merge(ratings, user, how='left', left_on='user_id', right_on='user_id')

In [0]:
def onehot(df,col):
  tmp = df[col].value_counts()
  tdict = tmp.to_dict()
  df[col+'_freq'] = df[col].map(tdict) 
  df = df.drop(col,axis=1)

  freqs= pd.cut(df[col+'_freq'],bins =([0] + np.percentile(tmp,[25,50,75,100]).tolist()),labels=['25%-','25-50%','50-75%','75+'])
  df = df.join(pd.get_dummies(freqs,prefix=col+'_freq'))
  df = df.drop(col+'_freq',axis=1)

  return df

In [0]:
ratings = onehot(ratings,'user_id')
ratings = onehot(ratings,'business_id')

In [0]:
ratings = ratings.join(pd.get_dummies(ratings.state))

In [0]:
ratings.drop(['text', 'review_id', 'date','state'], axis=1, inplace=True)

In [15]:
ratings.head()

,funny,useful,stars,cool,TrainTest,review_count,average_stars,user_id_freq_25%-,user_id_freq_25-50%,user_id_freq_50-75%,user_id_freq_75+,business_id_freq_25%-,business_id_freq_25-50%,business_id_freq_50-75%,business_id_freq_75+,AB,AZ,FL,IL,NC,NV,OH,ON,PA,QC,SC,WI
0,0,0,2.0,0,1,249,3.22,0,0,1,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0
1,0,2,4.0,0,1,136,3.82,0,1,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0
2,0,2,4.0,2,1,55,3.64,0,1,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0
3,4,1,4.0,1,1,178,3.42,0,0,0,1,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0
4,0,0,5.0,0,1,43,4.07,0,1,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0


In [0]:
train_X = ratings.drop(targetColumn, axis=1).loc[ratings.TrainTest == 1]
train_y = ratings[targetColumn].loc[ratings.TrainTest == 1]

test_X = ratings.drop(targetColumn, axis=1).loc[ratings.TrainTest == 0]
test_y = ratings[targetColumn].loc[ratings.TrainTest == 0]

In [0]:
train_X = train_X.drop('TrainTest',axis=1)
test_X = test_X.drop('TrainTest',axis=1)

In [0]:
tr_X = train_X.to_numpy()
tr_y = train_y.to_numpy()
te_X = test_X.to_numpy()
te_y = test_y.to_numpy()
X_tr_sparse = sp.csr_matrix(tr_X)
X_te_sparse = sp.csr_matrix(te_X)

In [19]:
from tffm import TFFMRegressor
from sklearn.metrics import mean_squared_error

model = TFFMRegressor(
  order=3, 
  rank=10, 
  optimizer=tf.train.AdamOptimizer(learning_rate=0.0003), 
  n_epochs=1000, 
  batch_size=1024,
  init_std=0.001,
  input_type='sparse',
  reg=0.01,
  seed=100
)

model.fit(X_tr_sparse, tr_y, show_progress=True)
predictions = model.predict(X_te_sparse)
print('[order={}] accuracy: {}'.format(3, accuracy_score(te_y, predictions > 0)))
print(mean_squared_error(te_y, predictions))







Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where






100%|██████████| 1000/1000 [11:46<00:00,  1.42epoch/s]

[order=3] accuracy: 0.11131897047940137
1.22329549218008


In [0]:
!pip install tffm

In [22]:
print(mean_squared_error(te_y, predictions))

1.4964518611881041


In [0]:
pred_train_y = model.predict(X_tr_sparse)

In [0]:
pd.DataFrame(
    pred_train_y, 
    index=train_y.index, 
    columns=['stars']
).to_csv(
    filePath + 'tffm_trainPredict.csv', index=False
)

pd.DataFrame(
    predictions, 
    index=test_y.index, 
    columns=['stars']
).to_csv(
    filePath + 'tffm_testPredict.csv', index=False
  )